In [2]:
!pip install guidedlda

  Using cached guidedlda-2.0.0.dev22.tar.gz (2.1 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached numpy-1.19.5-cp36-cp36m-win_amd64.whl (13.2 MB)
  Created wheel for guidedlda: filename=guidedlda-2.0.0.dev22-cp36-cp36m-win_amd64.whl size=2307828 sha256=d03c660663da1eeb9e5e7276e49d00c99c2b768fb5f5c5c10a42b2ce3185d751
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\53\52\6f\9c699da745b09a6c38beb3386fbab23eb6e0353a053736ff82
Successfully built guidedlda
  Using cached guidedlda-2.0.0.dev22-cp36-cp36m-win_amd64.whl


In [22]:
!pip install gensim dataclasses

  Using cached dataclasses-0.8-py3-none-any.whl (19 kB)


In [53]:
!pip install sklearn

  Using cached sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-0.24.2-cp36-cp36m-win_amd64.whl (6.8 MB)


In [54]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import wordnet
from nltk.corpus import stopwords 
import numpy as np
from nltk.tokenize import word_tokenize
np.random.seed(42)
import nltk
nltk.download('omw-1.4')
import sklearn

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [49]:
stop_words = set(stopwords.words('english'))

### 데이터 불러오기

In [34]:
import pandas as pd

df = pd.read_csv('./raw_data.csv')
df.head()

,document,text
0,Rye2019_550250.txt,\nManagement's Discussion and Analysis Decembe...
1,Rye2020_550250.txt,"\n\nAs management of the City of Rye, New York..."
2,Rye2018_550250.txt,"\n\nAs management of the City of Rye, New York..."
3,Rye2013_550250.txt,"Management's Discussion and Analysis (""MD&A"")\..."
4,Rye2015_550250.txt,"\n\n\nAs management of the City of Rye, New Yo..."


In [50]:
def get_wordnet_pos(word):
    '''tags parts of speech to tokens
    Expects a string and outputs the string and 
    its part of speech'''
    
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def word_lemmatizer(text):
    '''lemamtizes the tokens based on their part of speech'''
    
    lemmatizer = WordNetLemmatizer()
    text = lemmatizer.lemmatize(text, get_wordnet_pos(text))
    return text

def reflection_tokenizer(text):
    
    text=re.sub(r'[\W_]+', ' ', text) #keeps alphanumeric characters
    text=re.sub(r'\d+', '', text) #removes numbers
    text = text.lower()
    tokens = [word for word in word_tokenize(text)]
    tokens = [word for word in tokens if len(word) >= 3]
    #removes smaller than 3 character
    tokens = [word_lemmatizer(w) for w in tokens]
    tokens = [s for s in tokens if s not in stop_words]
    return tokens

In [60]:
df['token'] = df['token'].map(lambda x:[w for w in x if not w in set(place_name)]) 

In [61]:
place_name= [
    'niagara', 'senecas', 'seneca', 'oneonta', 'depauw', 'saratoga', 'dunkirk', 'wastewater', 'buffalo', 'amsterdam', ' schenectady', 'newburgh', 'broadway', 'hudson', 'binghamton', 'broome', 'syracuse', 'canadaigua', 'tonawanda', 'ogdensburg', 'glens', 'falls', 'rochester', 'albany', 'ithaca', 'auburn', 'eastman', 'rochelle', 'yonkers'
]


In [51]:
df['token'] = df['text'].apply(reflection_tokenizer)


In [62]:
df

,document,text,token
0,Rye2019_550250.txt,\nManagement's Discussion and Analysis Decembe...,"[management, discussion, analysis, december, m..."
1,Rye2020_550250.txt,"\n\nAs management of the City of Rye, New York...","[management, city, rye, new, york, city, offer..."
2,Rye2018_550250.txt,"\n\nAs management of the City of Rye, New York...","[management, city, rye, new, york, city, offer..."
3,Rye2013_550250.txt,"Management's Discussion and Analysis (""MD&A"")\...","[management, discussion, analysis, management,..."
4,Rye2015_550250.txt,"\n\n\nAs management of the City of Rye, New Yo...","[management, city, rye, new, york, city, offer..."
...,...,...,...
263,SaratogaSprings2018_410252.txt,\n\nAs management of the City of Saratoga Spri...,"[management, city, spring, new, york, city, of..."
264,SaratogaSprings2017_410252.txt,\nManagement's Discussion and Analysis Decembe...,"[management, discussion, analysis, december, m..."
265,SaratogaSprings2019_410252.txt,\n\nAs management of the City of Saratoga Spri...,"[management, city, spring, new, york, city, of..."
266,SaratogaSprings2015_410252.txt,\n\nAs management of the City of Saratoga Spri...,"[management, city, spring, new, york, city, of..."


In [63]:
df.to_csv('./preprocessed_ver3.csv')

In [64]:
from sklearn.feature_extraction.text import CountVectorizer

In [81]:
#
token_vectorizer = CountVectorizer(tokenizer = reflection_tokenizer, min_df=10, stop_words=stop_words, ngram_range=(1, 2))
X = token_vectorizer.fit_transform(df.text)

C:\Users\Administrator\miniconda3\envs\guidedlda\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['far'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [82]:
X.shape

(268, 7906)

In [97]:
X= X.toarray().astype(int)

In [83]:
#word id 만들기
tf_feature_names = token_vectorizer.get_feature_names()
word2id = dict((v, idx) for idx, v in enumerate(tf_feature_names))

In [84]:
vocab_df=pd.DataFrame(word2id.keys())

vocab_df.to_csv('./guided_vocab_df.csvf', index=False)

In [124]:
model = guidedlda.GuidedLDA(n_topics=5, n_iter=1000, random_state=7, refresh=10)
model.fit(X)

INFO:guidedlda:n_documents: 268
INFO:guidedlda:vocab_size: 7906
INFO:guidedlda:n_words: 785357
INFO:guidedlda:n_topics: 5
INFO:guidedlda:n_iter: 1000
INFO:guidedlda:<0> log likelihood: -7249764
INFO:guidedlda:<10> log likelihood: -6695950
INFO:guidedlda:<20> log likelihood: -6401648
INFO:guidedlda:<30> log likelihood: -6320655
INFO:guidedlda:<40> log likelihood: -6284473
INFO:guidedlda:<50> log likelihood: -6263792
INFO:guidedlda:<60> log likelihood: -6248034
INFO:guidedlda:<70> log likelihood: -6235924
INFO:guidedlda:<80> log likelihood: -6225865
INFO:guidedlda:<90> log likelihood: -6217940
INFO:guidedlda:<100> log likelihood: -6213132
INFO:guidedlda:<110> log likelihood: -6206888
INFO:guidedlda:<120> log likelihood: -6203228
INFO:guidedlda:<130> log likelihood: -6201019
INFO:guidedlda:<140> log likelihood: -6199076
INFO:guidedlda:<150> log likelihood: -6196626
INFO:guidedlda:<160> log likelihood: -6195976
INFO:guidedlda:<170> log likelihood: -6195237
INFO:guidedlda:<180> log likeliho

In [125]:
n_top_words = 15
topic_word = model.topic_word_
for i, topic_dist in enumerate(topic_word):
     topic_words = np.array(tf_feature_names)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
     print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: fund increase decrease year city revenue debt activity service net capital financial project statement business
Topic 1: city fund statement activity year financial net position net position report increase asset total governmental revenue
Topic 2: million fund city statement balance financial year fund balance increase budget financial statement total fiscal revenue tax
Topic 3: city fund year financial statement revenue tax increase general budget cost capital new balance project
Topic 4: fund city financial statement financial statement governmental balance government year fund balance governmental fund net general information report


In [126]:
doc_topic = model.transform(X)
for i in range(268):
    print("doc_num : {} top topic: {} distribution : {} topic words: {} ".format(i, doc_topic[i].argmax(), doc_topic[i],', '.join(np.array(tf_feature_names)[list(reversed(X[i,:].argsort()))[0:5]])))

doc_num : 0 top topic: 4 distribution : [3.80175319e-01 4.89241920e-04 6.10206669e-02 1.92804597e-02
 5.39034312e-01] topic words: fund, city, statement, financial, year 
doc_num : 1 top topic: 4 distribution : [0.36685098 0.00070116 0.06803167 0.04116901 0.52324718] topic words: fund, city, statement, financial, revenue 
doc_num : 2 top topic: 4 distribution : [0.35641281 0.00153857 0.09711662 0.05382678 0.49110523] topic words: fund, city, statement, financial, year 
doc_num : 3 top topic: 4 distribution : [0.37018981 0.00370222 0.08792318 0.00607348 0.53211131] topic words: fund, city, statement, financial, year 
doc_num : 4 top topic: 4 distribution : [0.37702102 0.00058737 0.09348667 0.01931461 0.50959032] topic words: fund, city, statement, financial, increase 
doc_num : 5 top topic: 4 distribution : [3.76850914e-01 2.61130894e-04 9.16339041e-02 1.69799781e-03
 5.29556053e-01] topic words: fund, city, statement, financial, revenue 
doc_num : 6 top topic: 4 distribution : [0.38899

#### seed topic 추가

In [119]:
seed_topic_list = [['bankruptcy', 'refuse', 'unemployment', 'stress', 'damage'], [ 'reasonable' ,'commitment', 'exempt', 'fair', 'faith', 'secure', 'encourage', 'successful', 'good', 'success', 'education', 'justice']]

In [120]:
model = guidedlda.GuidedLDA(n_topics=2, n_iter=100, random_state=7, refresh=10)
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id
model.fit(X, seed_topics=seed_topics, seed_confidence=0.9)

INFO:guidedlda:n_documents: 268
INFO:guidedlda:vocab_size: 7906
INFO:guidedlda:n_words: 785357
INFO:guidedlda:n_topics: 2
INFO:guidedlda:n_iter: 100
INFO:guidedlda:<0> log likelihood: -6436580
INFO:guidedlda:<10> log likelihood: -6253993
INFO:guidedlda:<20> log likelihood: -6083769
INFO:guidedlda:<30> log likelihood: -6060945
INFO:guidedlda:<40> log likelihood: -6049809
INFO:guidedlda:<50> log likelihood: -6041739
INFO:guidedlda:<60> log likelihood: -6034428
INFO:guidedlda:<70> log likelihood: -6030583
INFO:guidedlda:<80> log likelihood: -6026152
INFO:guidedlda:<90> log likelihood: -6023703
INFO:guidedlda:<99> log likelihood: -6021055


In [121]:
n_top_words = 15
topic_word = model.topic_word_
for i, topic_dist in enumerate(topic_word):
     topic_words = np.array(tf_feature_names)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
     print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: fund city statement financial million financial statement governmental year balance increase government fund balance net general governmental fund
Topic 1: city fund year financial statement increase activity revenue net position asset tax include capital net position


In [122]:
doc_topic = model.transform(X)
doc_topic[0]

array([0.86710622, 0.13289378])

In [123]:
doc_topic = model.transform(X)
for i in range(268):
    print("doc_num : {} top topic: {} distribution : {} topic words: {} ".format(i, doc_topic[i].argmax(), doc_topic[i],', '.join(np.array(tf_feature_names)[list(reversed(X[i,:].argsort()))[0:5]])))

doc_num : 0 top topic: 0 distribution : [0.86710622 0.13289378] topic words: fund, city, statement, financial, year 
doc_num : 1 top topic: 0 distribution : [0.84513416 0.15486584] topic words: fund, city, statement, financial, revenue 
doc_num : 2 top topic: 0 distribution : [0.82317104 0.17682896] topic words: fund, city, statement, financial, year 
doc_num : 3 top topic: 0 distribution : [0.8859354 0.1140646] topic words: fund, city, statement, financial, year 
doc_num : 4 top topic: 0 distribution : [0.85253543 0.14746457] topic words: fund, city, statement, financial, increase 
doc_num : 5 top topic: 0 distribution : [0.88929515 0.11070485] topic words: fund, city, statement, financial, revenue 
doc_num : 6 top topic: 0 distribution : [0.85700739 0.14299261] topic words: fund, city, statement, financial, revenue 
doc_num : 7 top topic: 0 distribution : [0.85410237 0.14589763] topic words: fund, city, statement, increase, financial 
doc_num : 8 top topic: 1 distribution : [0.324868